In [34]:
using DualOptimalFiltering, Distributions, RCall, Random, KernelEstimator

R"library(tidyverse)"

RObject{StrSxp}
 [1] "dplyr"     "purrr"     "readr"     "tidyr"     "tibble"    "ggplot2"  
 [7] "tidyverse" "stats"     "graphics"  "grDevices" "utils"     "datasets" 
[13] "methods"   "base"     


In [35]:
function create_kde_fun(smp::Array{Float64,1})
    bw = bwlscv(smp, gammakernel)
    f(x::Float64) = kerneldensity(smp, xeval=Float64[x], lb=0, ub=Inf, kernel=gammakernel, h = bw)[1]
    f(x::Array{Float64, 1}) = kerneldensity(smp, xeval=x, lb=0, ub=Inf, kernel=gammakernel, h = bw)
    return f
end

create_kde_fun (generic function with 1 method)

In [36]:
Random.seed!(0)
α = 1.2
β = 6.
smp = rand(Gamma(α, 1/β), 10000)

10000-element Array{Float64,1}:
 0.23057979351846936 
 0.2607779452078286  
 0.07263439587742074 
 0.21280482588177607 
 0.16181186215188184 
 0.12589892728456487 
 0.10196011821122809 
 0.4394680075055839  
 0.04037786697435014 
 0.17867595157546096 
 0.25729062042251777 
 0.0733682209283637  
 0.7345562949118852  
 ⋮                   
 0.07939248325967642 
 0.15016532023499515 
 0.6429083171161857  
 0.18214052279496454 
 0.04283586999797066 
 0.0529020979991244  
 0.013001989333486176
 0.029859347809401464
 0.2878736409967021  
 0.4757573757993345  
 0.5301498042875699  
 0.23295419171310186 

In [37]:
α_list, β_list = DualOptimalFiltering.create_gamma_kde_mixture_parameters(smp)

([110.186, 124.486, 35.3945, 101.769, 77.6227, 60.6169, 49.2812, 209.101, 20.1201, 85.6084  …  305.436, 87.249, 21.2841, 26.0507, 7.15683, 15.1393, 137.317, 226.285, 252.042, 111.311], [473.53, 473.53, 473.53, 473.53, 473.53, 473.53, 473.53, 473.53, 473.53, 473.53  …  473.53, 473.53, 473.53, 473.53, 473.53, 473.53, 473.53, 473.53, 473.53, 473.53])

In [38]:
kde_dens = DualOptimalFiltering.create_gamma_mixture_density_αβ(α_list, β_list, repeat([1/length(α_list)], length(α_list)))

(::getfield(DualOptimalFiltering, Symbol("#res#4")){Array{Float64,1},Array{Float64,1},Array{Float64,1}}) (generic function with 1 method)

In [39]:
kde_dens_package = create_kde_fun(smp)

(::getfield(Main, Symbol("#f#3")){Array{Float64,1},Float64}) (generic function with 2 methods)

In [40]:
grid_x = range(0, stop = 1., length = 500)

0.0:0.002004008016032064:1.0

In [ ]:
R"tibble(grid = $grid_x, 
            exact = $(pdf.(Gamma(α, 1/β), grid_x)), 
            kde = $(kde_dens.(grid_x)),
            kde_package = $(kde_dens_package.(grid_x))) %>%
    gather(type, val, exact:kde) %>%
    ggplot(aes(x = grid, y = val, colour = type)) + 
    theme_minimal() +
    geom_line()"